In [2]:
import csv
import os
import json
from collections import defaultdict
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

# Initialize the LLM
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.7)


In [16]:
import csv
import os
import json
from collections import defaultdict
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

# Initialize the LLM
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.7)

def load_pricing_history(csv_file="./historical_pricing.csv"):
    """
    Loads historical pricing data from a CSV file with columns:
    Supplier, Year, Service, Price ($).
    Returns a nested dictionary formatted as:
    {
      <Service>: {
        "<Year>": {
          <Supplier>: <Price>
        }
      },
      ...
    }
    """
    pricing_data = defaultdict(lambda: defaultdict(dict))
    
    if not os.path.exists(csv_file):
        print(f"⚠️ Warning: {csv_file} not found!")
        return {}
    
    with open(csv_file, mode="r", newline="", encoding="utf-8") as file:
        reader = csv.DictReader(file)
        required_cols = ["Supplier", "Year", "Service", "Price ($)"]
        missing_cols = [col for col in required_cols if col not in reader.fieldnames]
        if missing_cols:
            print(f"⚠️ Warning: Missing columns in {csv_file}: {missing_cols}")
            return {}

        for row in reader:
            supplier = row["Supplier"].strip()
            year = row["Year"].strip()
            service = row["Service"].strip()
            
            try:
                price = float(row["Price ($)"].replace(',', ''))
            except ValueError:
                print(f"⚠️ Warning: Non-numeric price in row: {row}")
                continue

            pricing_data[service][year][supplier] = price
    
    return dict(pricing_data)

def generate_pricing_risk_report(pricing_data: dict):
    """
    Uses an LLM to analyze pricing history and generate a risk report.
    """
    context = json.dumps(pricing_data, indent=2)

    response_schemas = [
        ResponseSchema(name="pricing_risk_report", description="A structured markdown report analyzing pricing trends, risks, and strategic recommendations.")
    ]

    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
    format_instructions = output_parser.get_format_instructions()

    prompt_template = PromptTemplate(
        input_variables=["context"],
        template="""
        You are an **AI Financial Risk Analyst** specializing in **pricing strategy and risk assessment**.
        Based on the **historical pricing data** provided below, generate a **comprehensive Pricing Risk Analysis Report**.

        **Guidelines:**
        - Identify **price volatility trends** (stable, fluctuating, extreme variations).
        - Detect **market risks** based on price fluctuations.
        - Assess **supplier pricing consistency** (high-risk suppliers with volatile pricing).
        - Predict **future pricing trends** based on historical patterns.
        - Offer **strategic procurement recommendations** (lock in contracts, diversify suppliers, negotiate better rates).
        - Present the analysis in a **detailed markdown report** format.

        ---
        ## 📊 **Pricing Risk Analysis Report**
        ### **1. Historical Price Trends**
        - Identify key **price movements** for each service.
        - Highlight **most volatile and most stable services**.

        ### **2. Market Risks & Opportunities**
        - Assess **inflationary risks**, seasonal patterns, and demand-driven pricing shifts.
        - Identify potential **opportunities for cost savings**.

        ### **3. Supplier Risk Assessment**
        - Evaluate **pricing consistency among suppliers**.
        - Identify **high-risk suppliers** due to frequent price fluctuations.

        ### **4. Future Price Predictions**
        - Forecast **expected price trends** for the next 4 quarters.
        - Provide confidence levels for predictions.

        ### **5. Strategic Recommendations**
        ✅ **Lock in contracts for stable prices?**
        ✅ **Diversify supplier base to reduce dependency?**
        ✅ **Negotiate better rates based on insights?**

        **Pricing Data:**
        ```json
        {context}
        ```

        {format_instructions}
        """,
        partial_variables={"format_instructions": format_instructions}
    )

    chain = prompt_template | llm | output_parser
    risk_report = chain.invoke({"context": context})
    
    return risk_report["pricing_risk_report"]

def test_pricing_risk_analysis(csv_file="./historical_pricing.csv"):
    """
    Function to test pricing risk analysis by loading data and generating a report.
    """
    pricing_history = load_pricing_history(csv_file)
    if not pricing_history:
        return "No valid pricing history found. Please check the input CSV."
    
    risk_report = generate_pricing_risk_report(pricing_history)
    return risk_report

In [17]:
pricing_history=load_pricing_history()

In [18]:
pricing_history

{'Cloud Storage': defaultdict(dict,
             {'2020': {'Supplier A': 90556.0,
               'Supplier B': 87605.0,
               'Supplier C': 85777.0,
               'Supplier D': 87975.0,
               'Supplier E': 93760.0},
              '2021': {'Supplier A': 98021.0,
               'Supplier B': 95575.0,
               'Supplier C': 98341.0,
               'Supplier D': 88890.0,
               'Supplier E': 96332.0},
              '2022': {'Supplier A': 102532.0,
               'Supplier B': 101863.0,
               'Supplier C': 94377.0,
               'Supplier D': 96539.0,
               'Supplier E': 99762.0},
              '2023': {'Supplier A': 101062.0,
               'Supplier B': 102981.0,
               'Supplier C': 95283.0,
               'Supplier D': 92341.0,
               'Supplier E': 97696.0},
              '2024': {'Supplier A': 92798.0,
               'Supplier B': 95974.0,
               'Supplier C': 96564.0,
               'Supplier D': 105045.0,
   

In [19]:
report=test_pricing_risk_analysis()

In [21]:
print(report)

# 📊 Pricing Risk Analysis Report

## **1. Historical Price Trends**
### **Price Movements**
- **Cloud Storage**: Prices have consistently increased from 2020 to 2024, with some fluctuations among suppliers.
- **Compute Instances**: A general upward trend with some consistent pricing among suppliers, yet with minor fluctuations.
- **AI-Powered Analytics**: Significant growth observed, with a notable peak in 2022 and slight adjustments in subsequent years.
- **Security & Compliance**: Prices have shown moderate fluctuations with a slight upward trend.
- **Hybrid Cloud Management**: Prices have shown the most fluctuation among suppliers, particularly in 2024.

### **Volatility Assessment**
- **Most Volatile Services**: Hybrid Cloud Management
- **Most Stable Services**: Compute Instances

## **2. Market Risks & Opportunities**
### **Assessment of Risks**
- **Inflationary Risks**: General increase in prices due to inflation trends observed in technology services.
- **Seasonal Patterns**: N